In [1]:
from glob import glob
from tqdm import tqdm
import os
import json
import cv2
import numpy as np
from pathlib import Path
from prettyprinter import cpprint, set_default_style

# pred_json_dirA = "./output/structure_result/test_A-pred.json_0702_91.130"
pred_json_dirA = "./output/structure_result/test_A-pred.json_0704_91.935"
pred_json_dirB = "./output/structure_result/test_A-pred.json"
visualize_dirA = f"{pred_json_dirA}_visualize"
visualize_dirB = f"{pred_json_dirB}_visualize"
diff_save_dir = "./output/structure_result/diff_map"

Path(diff_save_dir).mkdir(parents=True, exist_ok=True)

for pred_json_path_A in tqdm(glob(os.path.join(pred_json_dirA, "*-pred.json"))):
    json_file = os.path.basename(pred_json_path_A)
    img_file = json_file.replace("-pred.json", ".jpg")
    img_id = img_file.split(".")[0]
    dataA = json.load(open(pred_json_path_A, "r"))
    dataB = json.load(open(os.path.join(pred_json_dirB, json_file), "r"))

    img_path_A = os.path.join(visualize_dirA, f"{img_id}.jpg")
    img_path_B = os.path.join(visualize_dirB, f"{img_id}.jpg")

    # 比较layout排布
    if dataA['layout'] != dataB['layout']:
        row = max(len(dataA['layout']), len(dataB['layout']))
        col = max(len(dataA['layout'][0]), len(dataB['layout'][0]))
        diff_map = [[0 for i in range(col)] for j in range(row)]
        for i in range(row):
            for j in range(col):
                if i >= len(dataA['layout']) or j >= len(dataA['layout'][0]):
                    diff_map[i][j] = 1
                elif i >= len(dataB['layout']) or j >= len(dataB['layout'][0]):
                    diff_map[i][j] = 1
                else:
                    if dataA['layout'][i][j] != dataB['layout'][i][j]:
                        diff_map[i][j] = 1
        imgA = cv2.imread(img_path_A)
        imgB = cv2.imread(img_path_B)
        widthA, heightA = imgA.shape[1], imgA.shape[0]
        widthB, heightB = imgB.shape[1], imgB.shape[0]
        if widthA != widthB:
            imgB = cv2.resize(imgA, (widthB, heightB))
        image = np.concatenate([imgA, imgB], axis=1)
        cv2.imwrite(os.path.join(diff_save_dir, f"{img_id}.jpg"), image)
        # cpprint(diff_map)


100%|██████████| 5187/5187 [11:41<00:00,  7.39it/s]
